In [11]:
pip install gspread oauth2client pandas google-auth google-auth-oauthlib google-auth-httplib2


Note: you may need to restart the kernel to use updated packages.


In [21]:
import datetime
import requests
import os
import json
import pandas as pd
import pygsheets
import configparser
from authlib.integrations.requests_client import OAuth2Session
from google.auth.transport.requests import Request
from google_auth_oauthlib.flow import InstalledAppFlow
import pickle

# Load configuration from confi.txt
config = configparser.ConfigParser()
config.read("/Users/yashasvi.reddy/Documents/adobe-aep-api-main/confi.txt")  # Ensure the path is correct

client_id = config["oauths2s"]["client_id"]
client_secret = config["oauths2s"]["client_secret"]

# Authentication and API request setup
aep_scopes = (
    "openid, AdobeID, read_organizations, additional_info.projectedProductContext, session"
)
token_endpoint = "https://ims-na1.adobelogin.com/ims/token/v3"
org_id = "10D31225525FF5790A490D4D@AdobeOrg"

sandbox = "pplus-prod"

client = OAuth2Session(client_id=client_id, client_secret=client_secret, scope=aep_scopes)
token = client.fetch_token(token_endpoint, grant_type="client_credentials")
access_token = token.get("access_token")

url = "https://platform.adobe.io/data/core/ups/segment/definitions"
all_segments = []
page = 0

while True:
    params = {"limit": 100, "orderBy": "desc:created", "page": page}
    r = requests.get(
        url=url,
        params=params,
        headers={
            "Authorization": f"Bearer {access_token}",
            "x-api-key": client_id,
            "x-gw-ims-org-id": org_id,
            "x-sandbox-name": sandbox,
            "accept-encoding": "gzip",
            "Accept": "application/json",
        },
    )
    
    if r.status_code == 200:
        res = r.json()
        if 'segments' in res and len(res['segments']) > 0:
            all_segments.extend(res['segments'])
            if len(res['segments']) < 100:
                break
            page += 1
        else:
            break
    else:
        print("Error code", r.status_code)
        print("Error Description", r.json())
        break

# Save data to a Google Sheet
df = pd.DataFrame(all_segments)

# Path to the client_secret.json file
client_secret_path = '/Users/yashasvi.reddy/Documents/adobe-aep-api-main/client_secret_58196927069-pt0646otevl7unvkhoc0n5qce21aup3m.apps.googleusercontent.com.json'

# Authenticate with Google Sheets
SCOPES = ['https://www.googleapis.com/auth/drive', 'https://www.googleapis.com/auth/spreadsheets']

creds = None
if os.path.exists('token.pickle'):
    with open('token.pickle', 'rb') as token:
        creds = pickle.load(token)
if not creds or not creds.valid:
    if creds and creds.expired and creds.refresh_token:
        creds.refresh(Request())
    else:
        flow = InstalledAppFlow.from_client_secrets_file(client_secret_path, SCOPES)
        creds = flow.run_local_server(port=0)
    with open('token.pickle', 'wb') as token:
        pickle.dump(creds, token)

gc = pygsheets.authorize(custom_credentials=creds)

# Create a new Google Sheet
spreadsheet_name = f"Segment Definitions Data {datetime.datetime.now().strftime('%Y-%m-%d %H-%M-%S')}"
sh = gc.create(spreadsheet_name)
sh.share('yashasvi.reddy@paramount.com', role='writer')  # Share with your email for access

# Select the first sheet
worksheet = sh.sheet1

# Write the DataFrame to the Google Sheet
worksheet.set_dataframe(df, (1, 1))

print(f"Data has been exported to new Google Sheet: {spreadsheet_name}")


Data has been exported to new Google Sheet: Segment Definitions Data 2024-08-06 18-44-34
